In [39]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch.optim as optimizer
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchsummary

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

VGG Implementation

In [41]:
class VGG(nn.Module):
  def __init__(self, select_type, output_dim):
    super().__init__()
    self.select_type = select_type
    self.avgpool = nn.AdaptiveAvgPool2d(7)
    self.classifier = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, output_dim)
    )

  def forward(self, x):
    x = self.select_type(x)
    # print(x.size())
    x = self.avgpool(x)
    # print(x.size())
    x = x.view(x.shape[0], -1)
    # print(x.size())
    x = self.classifier(x)

    return x

In [42]:
vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
vgg13_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
vgg16_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512,
                'M', 512, 512, 512, 'M']
vgg19_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512,
                512, 512, 'M', 512, 512, 512, 512, 'M']

In [43]:
def select_vgg_layers(config, batch_norm):
  layers = []
  in_channels = 3

  for c in config:
    assert c == 'M' or isinstance(c, int)

    if c == 'M':
      layers += [nn.MaxPool2d(kernel_size = 2)]

    else:
      conv2d = nn.Conv2d(in_channels, c, kernel_size=3, padding=1)
      if batch_norm :
        layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace=True)]
      else :
        layers += [conv2d, nn.ReLU(inplace=True)]
      in_channels = c

  return nn.Sequential(*layers)

In [44]:
model = VGG(select_vgg_layers(vgg11_config, batch_norm=True), 10).to(device)
torchsummary.summary(model, input_size=(3, 224, 224),batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [64, 64, 224, 224]           1,792
       BatchNorm2d-2         [64, 64, 224, 224]             128
              ReLU-3         [64, 64, 224, 224]               0
         MaxPool2d-4         [64, 64, 112, 112]               0
            Conv2d-5        [64, 128, 112, 112]          73,856
       BatchNorm2d-6        [64, 128, 112, 112]             256
              ReLU-7        [64, 128, 112, 112]               0
         MaxPool2d-8          [64, 128, 56, 56]               0
            Conv2d-9          [64, 256, 56, 56]         295,168
      BatchNorm2d-10          [64, 256, 56, 56]             512
             ReLU-11          [64, 256, 56, 56]               0
           Conv2d-12          [64, 256, 56, 56]         590,080
      BatchNorm2d-13          [64, 256, 56, 56]             512
             ReLU-14          [64, 256,

In [45]:
model = VGG(select_vgg_layers(vgg16_config, batch_norm=True), 10).to(device)
torchsummary.summary(model, input_size=(3, 224, 224),batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [64, 64, 224, 224]           1,792
       BatchNorm2d-2         [64, 64, 224, 224]             128
              ReLU-3         [64, 64, 224, 224]               0
            Conv2d-4         [64, 64, 224, 224]          36,928
       BatchNorm2d-5         [64, 64, 224, 224]             128
              ReLU-6         [64, 64, 224, 224]               0
         MaxPool2d-7         [64, 64, 112, 112]               0
            Conv2d-8        [64, 128, 112, 112]          73,856
       BatchNorm2d-9        [64, 128, 112, 112]             256
             ReLU-10        [64, 128, 112, 112]               0
           Conv2d-11        [64, 128, 112, 112]         147,584
      BatchNorm2d-12        [64, 128, 112, 112]             256
             ReLU-13        [64, 128, 112, 112]               0
        MaxPool2d-14          [64, 128,

In [46]:
train_transforms = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.RandomRotation(5),
                    transforms.RandomHorizontalFlip(0.5),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])
                    ])

test_transforms = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229,0.224, 0.225])
])

In [47]:
cifar10_train = datasets.CIFAR10(root="../Data/", train=True, transform=train_transforms, download=True)
trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=64,shuffle=True, num_workers=2)


cifar10_test = datasets.CIFAR10(root="../Data/", train=False, transform=test_transforms, download=True)
testloader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

images, labels = next(iter(trainloader))

print(images.size(), labels.size())

Files already downloaded and verified
Files already downloaded and verified
torch.Size([64, 3, 224, 224]) torch.Size([64])


In [48]:
model = VGG(select_vgg_layers(vgg11_config, batch_norm=True), 10).to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-7)

에러 값이 오른다... 추후에 다시 손 봐야할 것 같아 어디 부분이 잘못되었는지

모델 구현은 얼추 맞는거 같다.

In [49]:
for i in range(3):
    for j,[image,label] in enumerate(trainloader):
        x = image.to(device)
        y= label.to(device)

        optimizer.zero_grad()
        loss = model.forward(x)
        loss = torch.Tensor.mean(loss)
        loss.backward()
        optimizer.step()

    print(loss)

tensor(-5.7636, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-21.0568, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-47.3666, device='cuda:0', grad_fn=<MeanBackward0>)
